In [25]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

In [26]:
!pip show langgraph

Name: langgraph
Version: 0.1.15
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: C:\Users\ujjwa\anaconda3\envs\qiskit_env\Lib\site-packages
Requires: langchain-core
Required-by: 


In [27]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from langchain.prompts import PromptTemplate
# import mysql.connector

# def get_database_connection():
#     return mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Prajapati@123",
#         database="filmcity"
#     )

# def fetch_item_availability(item_name):
#     connection = get_database_connection()
#     cursor = connection.cursor(dictionary=True)
#     query = "SELECT availability, price FROM inventory WHERE item_name = %s"
#     cursor.execute(query, (item_name,))
#     result = cursor.fetchone()
#     cursor.close()
#     connection.close()
#     return result

# draft_writer_prompt = PromptTemplate(
#     template="""system
#     You are the Email Writer Agent. Take the INITIAL_EMAIL below from a human that has emailed our company email address, the email_category \
#     that the categorizer agent gave it, and the research from the research agent, and \
#     write a helpful email in a thoughtful and friendly way.

#     Categories and actions:
#     - If the email_category is 'inquiry', check the database for item availability:
#         - If available, reply with the item's price.
#         - If not available, suggest similar available items.
#     - If the email_category is 'review':
#         - For positive reviews, thank the sender and encourage them to share their experience on social media.
#         - For negative reviews, escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply.
#     - If the email_category is 'assistance_request', use a Retrieval-Augmented Generation (RAG) approach to search for solutions to reported equipment issues:
#         - If a suitable solution is found, provide it in the reply.
#         - If no solution is found, escalate the issue to customer service.
#     - If the email_category is 'general', forward the email to customer service for further evaluation.

#     You never make up information that hasn't been provided by the research_info or in the initial_email.
#     Always sign off the emails in an appropriate manner and from Sarah, the Resident Manager.

#     Return the email as JSON with a single key 'email_draft' and no preamble or explanation.

#     user
#     INITIAL_EMAIL: {initial_email} \n
#     EMAIL_CATEGORY: {email_category} \n
#     RESEARCH_INFO: {research_info} \n
#     assistant
#     """,
#     input_variables=["initial_email","email_category","research_info"],
# )

# def generate_email_response(initial_email, email_category, research_info):
#     if email_category == 'inquiry':
#         item_name = research_info.get('item_name')
#         item_info = fetch_item_availability(item_name)
#         if item_info:
#             if item_info['availability']:
#                 research_info['response'] = f"The item is available at a price of ${item_info['price']}."
#             else:
#                 research_info['response'] = "The item is currently not available. Here are some similar items..."
#         else:
#             research_info['response'] = "The item is not found in our inventory."

#     # Call the prompt with the updated research_info
#     email_draft = draft_writer_prompt(initial_email=initial_email, email_category=email_category, research_info=research_info)
#     return email_draft['email_draft']

# # Example usage
# initial_email = "Hi, I would like to know the availability and price of the Sony A7III camera."
# email_category = "inquiry"
# research_info = {"item_name": "Sony A7III"}

# email_response = generate_email_response(initial_email, email_category, research_info)
# print(email_response)


In [28]:
!mysql -h localhost -u root -p

'mysql' is not recognized as an internal or external command,
operable program or batch file.


# The goal

Reply to a customer email
1. get the email
2. categorize the email as a "sales", "custom enquiry", "off topic", "customer complaint"
3. use the category & initial email to create keywords for a search to research info needed for the reply
4. write a draft reply
5. check the reply
7. rewrite if needed



In [29]:
import os
import getpass
# from google.colab import userdata

GROQ_API_KEY = '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'
TAVILY_API_KEY = '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'

# os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [30]:
### Tracing (optional)
import os
from pprint import pprint

LANGCHAIN_API_KEY = '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'

os.environ['LANGCHAIN TRACING V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN API KEY'] = LANGCHAIN_API_KEY

In [31]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
    model="llama3-70b-8192"
)

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser


### Utils

In [33]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
    """
  with open(f"{filename}.md", "w") as f:
    f.write(content)

## Basic Chains

1. Categorize EMAIL  
2. Research Router  
3. Search Keywords  
4. Write Draft Email  
5. Rewrite Router  
6. Draft Email Analysis  
7. Rewrite Email

In [34]:
# Categorize Email
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_enquiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'customer_feedback'


In [35]:
### Modified Categorize Email
# from langchain import PromptTemplate, GROQ_LLM, StrOutputParser

# Categorize Email
prompt = PromptTemplate(
    template="""system
    You are an Email Categorizer Agent. You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way.

    user
    Conduct a comprehensive analysis of the email provided and categorize it into one of the following categories:
        inquiry - used when someone is asking for information about film equipment availability or pricing.
        review - used when someone is giving feedback about the service or equipment.
        assistance_request - used when someone is asking for help or support regarding equipment usage or issues.
        general - used when the email does not fit into any of the above categories.

    Output a single category only from the types ('inquiry', 'review', 'assistance_request', 'general').
    eg:
    'inquiry'

    EMAIL CONTENT:\n\n {initial_email} \n\n

    assistant
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)


'review'


In [36]:
## Research Router
research_route_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the initial email and routing web search or directly to a draft email. \n

    Use the following criteria to decide how to route the email: \n\n

    If the initial email only requires a simple response
    Just choose 'draft_email'  for questions you can easily answer, prompt engineering, and adversarial attacks.
    If the email is just saying thank you etc then choose 'draft_email'

    You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use research-info.
    Give a binary choice 'research_info' or 'draft_email' based on the question. Return the a JSON with a single key 'router_decision' and
    no premable or explaination. use both the initial email and the email category to make your decision
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Email to route INITIAL_EMAIL : {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

research_router = research_route_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'

print(research_router.invoke({"initial_email": EMAIL, "email_category": email_category}))

{'router_decision': 'draft_email'}


In [37]:
## Search keywords
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    given the INITIAL_EMAIL and EMAIL_CATEGORY. Work out the best keywords that will find the best
    info for helping to write the final email.

    Return a JSON with a single key 'keywords' with no more than 3 keywords and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(search_keyword_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'keywords': ['hotel customer feedback examples', 'positive feedback response', 'resort customer appreciation']}


In [38]:
## Write Draft Email
draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent take the INITIAL_EMAIL below  from a human that has emailed our company email address, the email_category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category, "research_info": research_info}))


{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful experience at our resort! We're thrilled to hear that our staff made a positive impact on your stay. Your kind words mean the world to us, and we'll be sure to pass them along to our team.\n\nWe value your feedback and appreciate your loyalty. We're always striving to improve and provide the best experience possible for our guests. If you have any suggestions or ideas on how we can continue to exceed your expectations, please don't hesitate to share them with us.\n\nOnce again, thank you for your kind words and for choosing to stay with us. We can't wait to welcome you back again soon!\n\nBest regards,\nSarah, Resident Manager"}


In [44]:
from langchain.prompts import PromptTemplate
import mysql.connector

def get_database_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Prajapati@123",
        database="filmcity"
    )

def fetch_item_availability(item_name):
    connection = get_database_connection()
    cursor = connection.cursor(dictionary=True)
    query = "SELECT StockQuantity, Price FROM products WHERE ProductName = %s"
    cursor.execute(query, (item_name,))
    result = cursor.fetchone()
    cursor.close()
    connection.close()
    return result

draft_writer_prompt = PromptTemplate(
    template="""system
    You are the Email Writer Agent. Take the INITIAL_EMAIL below from a human that has emailed our company email address, the email_category \
    that the categorizer agent gave it, and the research from the research agent, and \
    write a helpful email in a thoughtful and friendly way.

    Categories and actions:
    - If the email_category is 'inquiry', check the database for item availability:
        - If available, reply with the item's price.
        - If not available, suggest similar available items.
    - If the email_category is 'review':
        - For positive reviews, thank the sender and encourage them to share their experience on social media.
        - For negative reviews, escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply.
    - If the email_category is 'assistance_request', use a Retrieval-Augmented Generation (RAG) approach to search for solutions to reported equipment issues:
        - If a suitable solution is found, provide it in the reply.
        - If no solution is found, escalate the issue to customer service.
    - If the email_category is 'general', forward the email to customer service for further evaluation.

    You never make up information that hasn't been provided by the research_info or in the initial_email.
    Always sign off the emails in an appropriate manner and from Sarah, the Resident Manager.

    Return the email as JSON with a single key 'email_draft' and no preamble or explanation.

    user
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    assistant
    """,
    input_variables=["initial_email","email_category","research_info"],
)

def generate_language_model_response(prompt):
    # Assuming you have a function to get a response from the language model
    # Here is a mock response for demonstration purposes
    return {"email_draft": "Dear Customer, ..."}

def generate_email_response(initial_email, email_category, research_info):
    if email_category == 'inquiry':
        item_name = research_info.get('item_name')
        item_info = fetch_item_availability(item_name)
        if item_info:
            if item_info['StockQuantity']:
                research_info['response'] = f"The item is available at a price of ${item_info['Price']}."
            else:
                research_info['response'] = "The item is currently not available. Here are some similar items..."
        else:
            research_info['response'] = "The item is not found in our inventory."

    # Generate the prompt using the format method
    prompt = draft_writer_prompt.format(initial_email=initial_email, email_category=email_category, research_info=research_info)
    
    # Get the response from the language model
    email_draft = generate_language_model_response(prompt)
    
    return email_draft['email_draft']

# Example usage
initial_email = "Hi, I would like to know the availability and price of the Sony A7III camera."
email_category = "inquiry"
research_info = {"item_name": "Sony A7III"}

email_response = generate_email_response(initial_email, email_category, research_info)
print(email_response)


Dear Customer, ...


In [45]:
## Rewrite Router
rewrite_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at evaluating the emails that are draft emails for the customer and deciding if they
    need to be rewritten to be better. \n

    Use the following criteria to decide if the DRAFT_EMAIL needs to be rewritten: \n\n

    If the INITIAL_EMAIL only requires a simple response which the DRAFT_EMAIL contains then it doesn't need to be rewritten.
    If the DRAFT_EMAIL addresses all the concerns of the INITIAL_EMAIL then it doesn't need to be rewritten.
    If the DRAFT_EMAIL is missing information that the INITIAL_EMAIL requires then it needs to be rewritten.

    Give a binary choice 'rewrite' (for needs to be rewritten) or 'no_rewrite' (for doesn't need to be rewritten) based on the DRAFT_EMAIL and the criteria.
    Return the a JSON with a single key 'router_decision' and no premable or explaination. \
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","draft_email"],
)

rewrite_router = rewrite_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
draft_email = "Yo we can't help you, best regards Sarah"

print(rewrite_router.invoke({"initial_email": EMAIL, "email_category":email_category, "draft_email":draft_email}))

{'router_decision': 'rewrite'}


In [46]:
## Draft Email Analysis
draft_analysis_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Quality Control Agent read the INITIAL_EMAIL below  from a human that has emailed \
    our company email address, the email_category that the categorizer agent gave it and the \
    research from the research agent and write an analysis of how the email.

    Check if the DRAFT_EMAIL addresses the customer's issued based on the email category and the \
    content of the initial email.\n

    Give feedback of how the email can be improved and what specific things can be added or change\
    to make the email more effective at addressing the customer's issues.

    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the analysis a JSON with a single key 'draft_analysis' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n\n
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_analysis_chain = draft_analysis_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

email_analysis = draft_analysis_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email})

pprint(email_analysis)

{'draft_analysis': {'addressing_customer_issue': 'fail',
                    'changes_needed': 'Add a personalized greeting, '
                                      'acknowledge and thank the customer for '
                                      'their feedback, express appreciation '
                                      'for their stay, and consider adding a '
                                      'loyalty program or incentive.',
                    'improvement_feedback': 'The draft email does not '
                                            'acknowledge or respond to the '
                                            "customer's positive feedback. It "
                                            'seems dismissive and '
                                            'unprofessional. A more '
                                            'appropriate response would be to '
                                            'thank the customer for their '
                                          

In [47]:
## Rewrite Email with ANlysis
rewrite_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Agent read the email analysis below from the QC Agent \
    and use it to rewrite and improve the draft_email to create a final email.


    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the final email as JSON with a single key 'final_email' which is a string and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    DRAFT_EMAIL_FEEDBACK: {email_analysis} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email",
                     "email_category",
                     "research_info",
                     "email_analysis",
                     "draft_email",
                     ],
)

rewrite_chain = rewrite_email_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

final_email = rewrite_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email,
                                "email_analysis":email_analysis})

final_email['final_email']

"Dear valued customer, I wanted to take a moment to personally thank you for taking the time to share your feedback with us. We truly appreciate your loyalty and are grateful for the opportunity to have had you as our guest. We value your opinions and are always looking for ways to improve. As a token of our appreciation, we would like to offer you a loyalty program for your next visit. Please let us know if there's anything else we can do to make your next stay even more enjoyable. Best regards, Sarah"

### Tool Setup

In [48]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

### State

In [49]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [50]:
from typing_extensions import TypedDict
from typing import List

# State

class GraphState(TypedDict):
  """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """

  initial_email: str
  email_category: str
  draft_email: str
  final_email: str
  research_info: List[str]
  info_needed: bool
  num_steps: int
  draft_email_feedback: dict

## Nodes

1. categorize_email
2. research_info_search  
3. draft_email_writer  
4. analyze_draft_email  
5. rewrite_email  
6. no_rewrite  
7. state_printer


In [51]:
def categorize_email(state):
  """ take the initial email and categorize it"""
  print("---CATEGORIZING INITIAL EMAIL---")
  initial_email = state['initial_email']
  num_steps = int(state['num_steps'])
  num_steps += 1

  email_category = email_category_generator.invoke({"initial_email": initial_email})
  print(email_category)
  # save to local disk
  write_markdown_file(email_category, "email_category")

  return {"email_category": email_category, "num_steps":num_steps}

In [52]:

def research_info_search(state):

    print("---RESEARCH INFO SEARCHING---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"initial_email": initial_email,
                                            "email_category": email_category })
    keywords = keywords['keywords']
    # print(keywords)
    full_searches = []
    for keyword in keywords[:1]:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    print(type(full_searches))
    # write_markdown_file(full_searches, "research_info")
    return {"research_info": full_searches, "num_steps":num_steps}

In [53]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [54]:
def analyze_draft_email(state):
    print("---DRAFT EMAIL ANALYZER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email_feedback = draft_analysis_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email}
                                               )
    # print(draft_email)
    # print(type(draft_email))

    write_markdown_file(str(draft_email_feedback), "draft_email_feedback")
    return {"draft_email_feedback": draft_email_feedback, "num_steps":num_steps}

In [55]:
def rewrite_email(state):
    print("---ReWRITE EMAIL ---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    draft_email_feedback = state["draft_email_feedback"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    final_email = rewrite_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email,
                                                "email_analysis": draft_email_feedback}
                                               )

    write_markdown_file(str(final_email), "final_email")
    return {"final_email": final_email['final_email'], "num_steps":num_steps}

In [56]:
def no_rewrite(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    write_markdown_file(str(draft_email), "final_email")
    return {"final_email": draft_email, "num_steps":num_steps}

In [57]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"Info Needed: {state['info_needed']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

### Conditional Edges

In [58]:
def route_to_research(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]


    router = research_router.invoke({"initial_email": initial_email,"email_category":email_category })
    print(router)
    # print(type(router))
    print(router['router_decision'])
    if router['router_decision'] == 'research_info':
        print("---ROUTE EMAIL TO RESEARCH INFO---")
        return "research_info"
    elif router['router_decision'] == 'draft_email':
        print("---ROUTE EMAIL TO DRAFT EMAIL---")
        return "draft_email"

In [59]:
def route_to_rewrite(state):

    print("---ROUTE TO REWRITE---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]

    # draft_email = "Yo we can't help you, best regards Sarah"

    router = rewrite_router.invoke({"initial_email": initial_email,
                                     "email_category":email_category,
                                     "draft_email":draft_email}
                                   )
    print(router)
    print(router['router_decision'])
    if router['router_decision'] == 'rewrite':
        print("---ROUTE TO ANALYSIS - REWRITE---")
        return "rewrite"
    elif router['router_decision'] == 'no_rewrite':
        print("---ROUTE EMAIL TO FINAL EMAIL---")
        return "no_rewrite"

## Build the Graph

### Add Nodes

In [60]:

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email) # categorize email
workflow.add_node("research_info_search", research_info_search) # web search
workflow.add_node("state_printer", state_printer)
workflow.add_node("draft_email_writer", draft_email_writer)
workflow.add_node("analyze_draft_email", analyze_draft_email)
workflow.add_node("rewrite_email", rewrite_email)
workflow.add_node("no_rewrite", no_rewrite)



## Add Edges

In [61]:
workflow.set_entry_point("categorize_email")

workflow.add_conditional_edges(
    "categorize_email",
    route_to_research,
    {
        "research_info": "research_info_search",
        "draft_email": "draft_email_writer",
    },
)
workflow.add_edge("research_info_search", "draft_email_writer")


workflow.add_conditional_edges(
    "draft_email_writer",
    route_to_rewrite,
    {
        "rewrite": "analyze_draft_email",
        "no_rewrite": "no_rewrite",
    },
)
workflow.add_edge("analyze_draft_email", "rewrite_email")
workflow.add_edge("no_rewrite", "state_printer")
workflow.add_edge("rewrite_email", "state_printer")
workflow.add_edge("state_printer", END)

In [62]:
# Compile
app = workflow.compile()

In [63]:
# EMAIL = """HI there, \n
# I am emailing to find out the current price of Bitcoin. \n

# Can you please help me/

# Thanks,
# John
# """

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

EMAIL = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

# EMAIL = """HI there, \n
# Why can't I get to sing?

# Thanks,
# Ringo
# """

# EMAIL = """HI there, \n
# Thanks for confirming my booking

# Thanks,
# Ringo
# """

In [64]:
# run the agent
inputs = {"initial_email": EMAIL,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
'general'
---ROUTE TO RESEARCH---
{'router_decision': 'research_info'}
research_info
---ROUTE EMAIL TO RESEARCH INFO---
'Finished running: categorize_email:'
---RESEARCH INFO SEARCHING---
Arizona weather April
[Document(page_content="{'location': {'name': 'Arizona', 'region': 'Atlantida', 'country': 'Honduras', 'lat': 15.63, 'lon': -87.32, 'tz_id': 'America/Tegucigalpa', 'localtime_epoch': 1722189435, 'localtime': '2024-07-28 11:57'}, 'current': {'last_updated_epoch': 1722188700, 'last_updated': '2024-07-28 11:45', 'temp_c': 28.7, 'temp_f': 83.6, 'is_day': 1, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png', 'code': 1240}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 22, 'wind_dir': 'NNE', 'pressure_mb': 1012.0, 'pressure_in': 29.9, 'precip_mm': 0.93, 'precip_in': 0.04, 'humidity': 78, 'cloud': 55, 'feelslike_c': 33.6, 'feelslike_f': 92.5, 'windchill_c': 28.7, 'windchill_f': 83.6, 'heatindex_c': 33

In [65]:
output = app.invoke(inputs)

print(output['final_email'])

---CATEGORIZING INITIAL EMAIL---
'review'
---ROUTE TO RESEARCH---
{'router_decision': 'research_info'}
research_info
---ROUTE EMAIL TO RESEARCH INFO---
---RESEARCH INFO SEARCHING---
Arizona weather April
[Document(page_content="{'location': {'name': 'Arizona', 'region': 'Atlantida', 'country': 'Honduras', 'lat': 15.63, 'lon': -87.32, 'tz_id': 'America/Tegucigalpa', 'localtime_epoch': 1722189435, 'localtime': '2024-07-28 11:57'}, 'current': {'last_updated_epoch': 1722188700, 'last_updated': '2024-07-28 11:45', 'temp_c': 28.7, 'temp_f': 83.6, 'is_day': 1, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png', 'code': 1240}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 22, 'wind_dir': 'NNE', 'pressure_mb': 1012.0, 'pressure_in': 29.9, 'precip_mm': 0.93, 'precip_in': 0.04, 'humidity': 78, 'cloud': 55, 'feelslike_c': 33.6, 'feelslike_f': 92.5, 'windchill_c': 28.7, 'windchill_f': 83.6, 'heatindex_c': 33.6, 'heatindex_f': 92.5, 'dewpoint_c': 

In [66]:
!cat /content/email_category.md

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [67]:
!cat /content/draft_email.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
